In [6]:
import pandas as pd
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import re

from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest


champion information

Data

In [7]:
path = "../../data/summoners_info/temp"
user = os.listdir(path)[1:][3]
path_user = path + '/' + user
# user
os.listdir(path_user)

['.DS_Store', 'most_pick.csv', 'solo_rank_games.csv', 'tier.csv']

In [8]:
user

'뽀뿌우'

In [9]:
solo_rank_games = pd.read_csv(path + '/' + user + '/solo_rank_games.csv', 
                              sep='\t', header=0)

solo_rank_games = solo_rank_games.replace(['패배','승리'], [0,1])
solo_rank_games

,Unnamed: 0,champion,ck_rate,cs_per_min,kda,result,timestamp
0,0,쓰레쉬,41,0.8,1.17,0,2017년 6월 19일 오후 3시 29분
1,1,카르마,44,0.4,1.56,1,2017년 6월 19일 오후 2시 57분
2,2,녹턴,67,4.2,1.43,0,2017년 6월 19일 오후 1시 42분
3,3,녹턴,100,5.0,1.43,1,2017년 6월 19일 오후 1시 08분
4,4,녹턴,47,6.2,1.75,0,2017년 6월 19일 오후 12시 50분
5,5,녹턴,65,4.3,2.17,0,2017년 6월 18일 오후 9시 29분
6,6,녹턴,65,4.6,2.50,0,2017년 6월 18일 오후 9시 00분
7,7,녹턴,32,5.8,5.50,1,2017년 6월 18일 오후 8시 20분
8,8,녹턴,64,3.4,0.88,0,2017년 6월 18일 오후 7시 42분
9,9,카르마,52,0.6,2.83,0,2017년 5월 17일 오후 8시 18분


In [10]:
d = []
t = []
s = solo_rank_games['timestamp']
for i in range(len(s)):
    ss = s[i]
    result1 = re.search('년 (.*)일', ss).group(1)
    result1 = result1.replace('월 ', '.')
    result2 = re.search('오(.*)분', ss).group(1)
    if result2[0] == '후':
        result2 = str(float(result2[2:].replace('시 ', '.')) + 12)
        if len(result2) > 4:
            result2 = result2[:5]
    else:
        result2 = str(result2[2:].replace('시 ', '.'))
    d.append(result1)
    t.append(result2)

solo_rank_games['date'] = d
solo_rank_games['time'] = t
# pd.to_datetime(solo_rank_games['time'], format='')
# solo_rank_games['time'] = pd.to_datetime(t, format='%H.%M')
solo_rank_games

,Unnamed: 0,champion,ck_rate,cs_per_min,kda,result,timestamp,date,time
0,0,쓰레쉬,41,0.8,1.17,0,2017년 6월 19일 오후 3시 29분,6.19,15.29
1,1,카르마,44,0.4,1.56,1,2017년 6월 19일 오후 2시 57분,6.19,14.57
2,2,녹턴,67,4.2,1.43,0,2017년 6월 19일 오후 1시 42분,6.19,13.42
3,3,녹턴,100,5.0,1.43,1,2017년 6월 19일 오후 1시 08분,6.19,13.08
4,4,녹턴,47,6.2,1.75,0,2017년 6월 19일 오후 12시 50분,6.19,24.5
5,5,녹턴,65,4.3,2.17,0,2017년 6월 18일 오후 9시 29분,6.18,21.29
6,6,녹턴,65,4.6,2.50,0,2017년 6월 18일 오후 9시 00분,6.18,21.0
7,7,녹턴,32,5.8,5.50,1,2017년 6월 18일 오후 8시 20분,6.18,20.2
8,8,녹턴,64,3.4,0.88,0,2017년 6월 18일 오후 7시 42분,6.18,19.42
9,9,카르마,52,0.6,2.83,0,2017년 5월 17일 오후 8시 18분,5.17,20.18


In [11]:
solo_rank_games.drop(['timestamp'], axis=1, inplace=True)

In [12]:
solo_rank_games 

solorank = solo_rank_games[['ck_rate', 'cs_per_min', 'kda']]
x = solorank
x

,ck_rate,cs_per_min,kda
0,41,0.8,1.17
1,44,0.4,1.56
2,67,4.2,1.43
3,100,5.0,1.43
4,47,6.2,1.75
5,65,4.3,2.17
6,65,4.6,2.50
7,32,5.8,5.50
8,64,3.4,0.88
9,52,0.6,2.83


In [13]:
tier = pd.read_csv(path + '/' + user + '/tier.csv', header=0, sep='\t')
tier[['month', 'tier']]

,month,tier
0,2016.11,S4-80LP
1,2016.12,B2-79LP
2,2017.01,S5-37LP
3,2017.02,G5-65LP
4,2017.03,G5-40LP
5,2017.04,G4-100LP
6,2017.05,P5-0LP
7,2017.06,G1-29LP


In [16]:
most_pick = pd.read_csv(path + '/' + user + '/most_pick2.csv', header=0, sep='\t').iloc[:5]


In [17]:
most_pick

,Unnamed: 0,avg_dmg_giv,avg_dmg_rec,champion,cs,gold,kda,most_death,most_kill,tower,win_ratio
0,0,"39,185","13,905",럭스,32.3,"9,172",1.94,18,14,0.43,39
1,1,"29,541","14,259",카르마,23.7,"8,526",2.01,12,6,0.32,40
2,2,"15,746","14,785",나미,11.3,"8,669",2.45,14,5,0.33,48
3,3,"53,022","18,128",미스 포츈,53.6,"10,144",1.56,15,16,0.63,44
4,4,"121,459","22,868",카사딘,175.6,"13,077",4.52,8,24,1.08,74


In [55]:
x

,ck_rate,cs_per_min,kda
0,41,0.8,1.17
1,44,0.4,1.56
2,67,4.2,1.43
3,100,5.0,1.43
4,47,6.2,1.75
5,65,4.3,2.17
6,65,4.6,2.50
7,32,5.8,5.50
8,64,3.4,0.88
9,52,0.6,2.83


In [56]:
outliers_fraction = 0.05

isolation forest

In [57]:

anomaly

,ck_rate,cs_per_min,kda
3,100,5.0,1.43
13,79,0.5,11.00
17,91,0.7,7.25
21,12,0.9,0.33
40,78,6.0,15.50
43,59,6.6,19.00
56,68,8.8,13.00
57,66,8.7,21.00
59,46,7.7,17.00
79,61,7.8,22.00


In [69]:
clf= IsolationForest(n_estimators=10, contamination=outliers_fraction)
clf.fit(x)
y = clf.predict(x)
a_index = np.where(y==-1)
anomaly = x.iloc[a_index]
solo_rank_games.iloc[anomaly.index]
a = solo_rank_games.iloc[anomaly.index]
a.loc[a['result'] == 1]

,Unnamed: 0,champion,ck_rate,cs_per_min,kda,result,date,time
3,3,녹턴,100,5.0,1.43,1,6.19,13.08
13,13,룰루,79,0.5,11.00,1,5.14,12.50
40,40,카사딘,78,6.0,15.50,1,5.9,22.5
43,43,카사딘,59,6.6,19.00,1,5.9,5.39
56,56,바루스,68,8.8,13.00,1,5.7,21.39
57,57,바루스,66,8.7,21.00,1,5.7,21.11
59,59,진,46,7.7,17.00,1,5.7,19.59
79,79,케넨,61,7.8,22.00,1,5.6,11.11
83,83,카직스,59,4.1,16.00,1,5.6,8.28
91,91,카직스,47,3.6,14.00,1,5.6,2.31


In [59]:
solo_rank_games

,Unnamed: 0,champion,ck_rate,cs_per_min,kda,result,date,time
0,0,쓰레쉬,41,0.8,1.17,0,6.19,15.29
1,1,카르마,44,0.4,1.56,1,6.19,14.57
2,2,녹턴,67,4.2,1.43,0,6.19,13.42
3,3,녹턴,100,5.0,1.43,1,6.19,13.08
4,4,녹턴,47,6.2,1.75,0,6.19,24.5
5,5,녹턴,65,4.3,2.17,0,6.18,21.29
6,6,녹턴,65,4.6,2.50,0,6.18,21.0
7,7,녹턴,32,5.8,5.50,1,6.18,20.2
8,8,녹턴,64,3.4,0.88,0,6.18,19.42
9,9,카르마,52,0.6,2.83,0,5.17,20.18


one class svm

In [60]:
clf = svm.OneClassSVM(nu=0.95 * outliers_fraction + 0.05, kernel="rbf", gamma=0.1)
clf.fit(x)
y = clf.predict(x)
a_index = np.where(y==-1)
anomaly = x.iloc[a_index]
anomaly
len(anomaly)

87

In [61]:
solo_rank_games.iloc[anomaly.index]
a = solo_rank_games.iloc[anomaly.index]
a.loc[a['result'] == 1]

,Unnamed: 0,champion,ck_rate,cs_per_min,kda,result,date,time
3,3,녹턴,100,5.0,1.43,1,6.19,13.08
17,17,룰루,91,0.7,7.25,1,5.12,14.48
18,18,룰루,65,0.7,7.50,1,5.12,14.04
36,36,진,59,6.3,13.00,1,5.10,21.28
41,41,카사딘,56,4.7,5.40,1,5.9,22.11
42,42,카사딘,66,5.4,7.00,1,5.9,6.28
48,48,바루스,48,6.2,7.00,1,5.9,2.43
52,52,럭스,39,0.6,5.00,1,5.8,22.38
79,79,케넨,61,7.8,22.00,1,5.6,11.11
93,93,바루스,47,6.8,6.67,1,5.6,1.02


covariance EllipticEnvelope

In [63]:
clf = EllipticEnvelope(contamination=outliers_fraction)
clf.fit(x)
y = clf.predict(x)
an_index = np.where(y==-1)
anomaly = x.iloc[an_index]
solo_rank_games.iloc[anomaly.index]
a = solo_rank_games.iloc[anomaly.index]
a.loc[a['result'] == 1]

,Unnamed: 0,champion,ck_rate,cs_per_min,kda,result,date,time
36,36,진,59,6.3,13.0,1,5.10,21.28
40,40,카사딘,78,6.0,15.5,1,5.9,22.5
43,43,카사딘,59,6.6,19.0,1,5.9,5.39
57,57,바루스,66,8.7,21.0,1,5.7,21.11
59,59,진,46,7.7,17.0,1,5.7,19.59
67,67,그레이브즈,52,7.2,17.0,1,5.7,4.08
79,79,케넨,61,7.8,22.0,1,5.6,11.11
83,83,카직스,59,4.1,16.0,1,5.6,8.28
91,91,카직스,47,3.6,14.0,1,5.6,2.31
98,98,코그모,38,5.8,15.0,1,5.4,6.18


In [65]:
user

'뽀뿌우'

In [67]:
months = ['2016.11', '2016.12', '2017.01', '2017.02', '2017.03', '2017.04', '2017.05', '2017.06']
tiers = ['S4-80LP', 'B2-79LP', 'S5-37LP', 'G5-65LP', 'G5-40LP', 'G4-100LP', 'P5-0LP', 'G1-29LP']

tier_graph = pd.DataFrame({
            'month': months,
            'tier': tiers
            })

tier_graph.to_csv(path + '/' + user + '/tier.csv', sep='\t', encoding='utf-8', header=True)